# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import Packages

In [2]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

### 1.2 Gather General Information

### 1.3 Create Databases